The first cells here are required to make the notebooks work without any installation in a browser. A normal notebook running on your personal machine would start later.

In [ ]:
%pip install ipyaladin
%pip install sidecar

In [ ]:
import micropip, http.client, pyodide_http
await micropip.install(["ssl","https://files.pythonhosted.org/packages/c3/c8/d74794dff5c6802994d3dfe506d691d9e0ff3e6b2f34df577899685b2617/astroquery-0.4.7.dev9008-py3-none-any.whl",
                        "https://rawcdn.githack.com/cds-astro/jupyterlite/cea7bf366dd077953acc578cbb63f2319793a018/content/pyodide/wheels/cdshealpix-0.6.5-cp311-cp311-emscripten_3_1_46_wasm32.whl",
                        "https://rawcdn.githack.com/cds-astro/jupyterlite/cea7bf366dd077953acc578cbb63f2319793a018/content/pyodide/wheels/mocpy-0.13.1-cp311-cp311-emscripten_3_1_46_wasm32.whl"])
pyodide_http.patch_all()
from importlib import reload
reload(http.client);

Here is the true beginning of the story.

# Introduction to ipyaladin
(and VizieR and MOCs)

In [ ]:
from ipyaladin import Aladin
from sidecar import Sidecar
from astroquery.vizier import Vizier
import astropy.units as u
from mocpy import MOC

In [ ]:
sidecar = Sidecar(title='aladin')
aladin = Aladin(fov=200,
                target='galactic center',
                coo_frame="J2000d",
                show_fullscreen_control=True,
                height=1000)
with sidecar:
    display(aladin)

In [ ]:
aladin.target = "M51"

## 1. Let's find the catalog identifier of the Dark Energy Survey in Vizier

In [ ]:
list_catalogs = Vizier.find_catalogs(["Dark Energy Survey", "Data Release 2"])
[print(catalog, description.description) for catalog, description in list_catalogs.items()];

The identifier is `II/371`

In [ ]:
catalog = "II/371"

## 2. A glimpse at its coverage

In [ ]:
catalog_coverage = MOC.from_fits(f"https://cdsarc.cds.unistra.fr/viz-bin/moc/{catalog}")

In [ ]:
aladin.add_moc_from_dict(catalog_coverage.serialize("json"),
                         {"color": "lightblue",
                          "fill": True,
                          "edge": True,
                          "opacity": 0.2
                         })

## 3. Grab sources in a cone

In [ ]:
cone = Vizier(catalog=catalog,
              row_limit=-1,
              columns=["RA_ICRS", "DE_ICRS", "gmag", "rmag", "imag", "DES"]
             ).query_region("15.3856880 -54.4340745", radius=1 * u.arcmin, frame="icrs")

In [ ]:
cone

In [ ]:
cone[0]

In [ ]:
aladin.add_table(cone[0])

In [ ]:
aladin.target = "15.3856880 -54.4340745"